In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
!pip install imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
!pip install factor-analyzer
from factor_analyzer import FactorAnalyzer
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('data_dummies.csv')
df

# Regression logistique de la maladie sur l'âge.
Il semblait assez évident que l'âge jouerait un rôle déterminant pour la contraction de maladies mais il conevnait de voir "à quel point". 
C'est pourquoi nous avons mené une régression logistique sur la question. 
Après plusieurs tentatives, c'est avec une méthode de ré échantillonnage et de changements de poids que nous sommes parvenus au résultat le plus précis. 
En effet, les malades atteints de cancer étant en minorité dans la population, sans modification de leur poids ou échantillonnage différent, il aurait été naturel de classer tous les individus comme bien portants pour minimiser le risque d'erreur.

In [ ]:
df_regression = df[['indicatrice__Cancers', 'classe_age']].dropna()


X = df_regression[['classe_age']]
y = df_regression['indicatrice__Cancers']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#re echantillonnage 
rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

#On change les poids
model = LogisticRegression(class_weight='balanced')
model.fit(X_resampled, y_resampled)
predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions, zero_division=1)

coefficients = model.coef_[0]
intercept = model.intercept_[0]

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')

print(f'Coefficients: {coefficients}')
print(f'Intercept: {intercept}')

On remarque que pour chaque classe d'âge franchie, les chances de contracter un cancer augmentent de 0.015, ce qui est un coefficient relativement important comparé aux résultats des autres régression.

In [ ]:


X_values = np.linspace(X.min(), X.max(), 1000).reshape(-1, 1)
probabilities = model.predict_proba(X_values)[:, 1]

plt.scatter(X, y, label='Données (peu évocateur)')
plt.plot(X_values, probabilities, color='red', label="Régression logistique de 'cancer' sur l'âge")
plt.xlabel('Âge')
plt.ylabel("Probabilité d'avoir un cancer")
plt.title('Courbe de régression')
plt.legend()
plt.show()

# CLUSTERING/ACP

In [ ]:
df_cluster = pd.read_csv('base_modelisation.csv')
df_cluster = df_cluster.drop(['Unnamed: 0', 'dept', 
                              'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'], axis = 1)
df_cluster = df_cluster.set_index('dep', drop = True)


In [ ]:
#On fait l acp et on standardise nos variables
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_cluster)
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=df_cluster.columns)
fig, ax = plt.subplots(figsize=(8, 8))

#Cercle de correlation
circle = plt.Circle((0, 0), 1, fill=False, edgecolor='b', linestyle='dashed')
ax.add_patch(circle)
for i, var in enumerate(loading_matrix.index):
    ax.arrow(0, 0, loading_matrix.loc[var, 'PC1'], loading_matrix.loc[var, 'PC2'],
             color='r', alpha=0.7, linewidth=0.5, head_width=0.02, head_length=0.02)
    ax.text(loading_matrix.loc[var, 'PC1'] * 1.15, loading_matrix.loc[var, 'PC2'] * 1.15, var,
            color='b', ha='center', va='center')

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_aspect('equal', 'box')
ax.set_title('Cercle de corrélations')

plt.show()

On remarque donc que la classe d'âge a une place unique dans la classification des départements.
Par ailleurs, on remarque que beaucoup de variables sont très liées entre elles, c'est pourquoi on effectue une ACP avec la méthode de rotation 'Promax' pour que ce soit plus interprétable et essayer de distinguer s'il n'y a pas d'autres facteurs qui peuvent être également importants.

In [ ]:
#Memes operations que precedemment mais avec la methode de rotation promax
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_cluster)
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
fa = FactorAnalyzer(rotation='promax', n_factors=2)
fa.fit(df_cluster)
rotated_loadings = fa.loadings_
fig, ax = plt.subplots(figsize=(8, 8))

# Cercle de correlations
circle = plt.Circle((0, 0), 1, fill=False, edgecolor='b', linestyle='dashed')
ax.add_patch(circle)
for i, var in enumerate(df_cluster.columns):
    ax.arrow(0, 0, rotated_loadings[i, 0], rotated_loadings[i, 1],
             color='r', alpha=0.7, linewidth=0.5, head_width=0.02, head_length=0.02)
    ax.text(rotated_loadings[i, 0] * 1.15, rotated_loadings[i, 1] * 1.15, var,
            color='b', ha='center', va='center')

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_aspect('equal', 'box')
ax.set_title('Circle of Correlation with Promax Rotation')

plt.show()

On remarque que les variables sont un (tout) petit peu moins groupées. Mais c'est pouruqoi, afin de trouver les variables les plus intéressantes, on garde celles qui contribuent le plus aux différentes composantes principales et on laisse de côté celles qui sont déjà très corrélées aux variables et qui, par ailleurs, contribuent moins aux composantes principales.

In [ ]:
list_to_drop = ['Energie', 'Autres transports international', 'Routier',
                'indicatrice_Traitements psychotropes (hors pathologies)',
                'indicatrice_Maladies cardio-neurovasculaires',
                'indicatrice_Maladies psychiatriques', 'indicatrice_Diabète','indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19',
                'indicatrice_Traitements du risque vasculaire (hors pathologies)']
                
                

In [ ]:
df_cluster_reduit = df_cluster.drop(list_to_drop, axis = 1)


In [ ]:
#on refait exactement les memes commandes que precemment mais avec un jeu de donnees reduit pour essayer
#d ameliorer notre modele
pca = PCA(n_components=2)
pca_result_bis = pca.fit_transform(df_cluster_reduit)
pca_df_reduit = pd.DataFrame(data=pca_result_bis, columns=['PC1', 'PC2'])
fa = FactorAnalyzer(rotation='promax', n_factors=2)
fa.fit(df_cluster_reduit)
rotated_loadings = fa.loadings_
fig, ax = plt.subplots(figsize=(8, 8))
circle = plt.Circle((0, 0), 1, fill=False, edgecolor='b', linestyle='dashed')
ax.add_patch(circle)
for i, var in enumerate(df_cluster_reduit.columns):
    ax.arrow(0, 0, rotated_loadings[i, 0], rotated_loadings[i, 1],
             color='r', alpha=0.7, linewidth=0.5, head_width=0.02, head_length=0.02)
    ax.text(rotated_loadings[i, 0] * 1.15, rotated_loadings[i, 1] * 1.15, var,
            color='b', ha='center', va='center')

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_aspect('equal', 'box')
ax.set_title('Cercle de corrélation avec moins de variables, et rotation Promax')

plt.show()

On remarque que mis à part le titre et une partie des variables assez liées en pollution, le cercle de corrélations est plus lisible. 
C'est pourquoi on va mener un clustering sur ces composantes principales.

### Clustering k-moyennes

In [ ]:
wcss = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(pca_df_reduit)
    wcss.append(kmeans.inertia_)

#On regarde la methode du coude
plt.plot(range(1, 11), wcss)
plt.title('Heuristique du coude')
plt.xlabel('Nombre de clusters')
plt.ylabel('Somme des carrés des distances intra cluster')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
df_cluster_reduit['Cluster'] = kmeans.fit_predict(pca_df)

In [ ]:
plt.scatter(pca_df_reduit['PC1'], pca_df_reduit['PC2'], c=df_cluster_reduit['Cluster'], cmap='viridis')
plt.title('Clusters faits avec les composantes principales')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
df_cluster_reduit[df_cluster_reduit['Cluster'] == 3]

On remarque que mis à part deux cas : Lozère et Territoire de Belfort, c'est surtout la classe d'âge et les maladies (qui sont très liées à l'âge) qui permettent de classer les départements.
Pour permettre de mieux voir ces résultats et de constater l'appartenance de chaque département à un certain cluster, nous faisons ce graphique 'interactif'.
Le clustering se base toujours sur l'ACP réduite avec la rotation promax. 
On continue de garder 5 clusters car c'est le nombre de clusters que nous avions retenus pour les cartes, et par ailleurs, la méthode du coude semble indiquer que c'est un bon résultat.

In [ ]:
df_cluster_reduit_reset = df_cluster_reduit.reset_index()

#ACP
pca = PCA(n_components=2)
pca_result_bis = pca.fit_transform(df_cluster_reduit_reset.drop(columns=['dep']))  
pca_df_reduit = pd.DataFrame(data=pca_result_bis, columns=['PC1', 'PC2'])
fa = FactorAnalyzer(rotation='promax', n_factors=2)
fa.fit(df_cluster_reduit_reset.drop(columns=['dep'])) 
rotated_loadings = fa.loadings_

#Clustering
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(pca_df_reduit)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Heuristique du coude')
plt.xlabel('Nombre de clusters')
plt.ylabel('Somme des carrés des distances intra-clusters')
plt.show()
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
df_cluster_reduit_reset['Cluster'] = kmeans.fit_predict(pca_df_reduit)
df_combined = pd.concat([df_cluster_reduit_reset[['dep', 'Cluster']], pca_df_reduit], axis=1)

# On utilise plotly pour que ce soit interactif
fig = px.scatter(df_combined, x='PC1', y='PC2', color='Cluster',
                 title='Clusters intéractifs', labels={'Cluster': 'Cluster'},
                 color_continuous_scale='viridis', hover_name='dep')

fig.show()

# ACP et Clustering sur la tranche des 30-60 ans 
Comme nous avons remarqué que l'âge avait une place prépondérante dans la détermination de la maladie, nous avons cherché sinon à éliminer l'effet de l'âge, du moins essayer de le minorer pour voir si la pollution avait en effet un impact sur la santé. 
La classe d'âge choisie est certes hétéroclite mais nous pensons que cela pourrait être plus évocateur. 

In [ ]:
df_cluster_30_60 = pd.read_csv('base_modelisation_30_60.csv')
df_cluster_30_60 = df_cluster_30_60.drop(['Unnamed: 0', 'dept', 
                              'indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'], axis = 1)
df_cluster_30_60 = df_cluster_30_60.set_index('dep', drop = True)

In [ ]:
print(df_cluster_30_60.var())

In [ ]:
#il y avait une erreur dans l acp reduite a une partie de la population
#on remarque qu'il y a des variables dont la variance est nulle
#on peut donc les supprimer comme elle n apportent pas d information pour categoriser la population

In [ ]:
df_cluster_30_60 = df_cluster_30_60.drop(['indicatrice_Autres affections de longue durée (dont 31 et 32)',
                               'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)'], axis = 1)


In [ ]:
pca_30_60 = PCA(n_components=2)
pca_result_30_60 = pca_30_60.fit_transform(df_cluster_30_60)
pca_df_30_60 = pd.DataFrame(data=pca_result_30_60, columns=['Custom_PC1', 'Custom_PC2'])  # Change variable names

# Promax rotation
fa = FactorAnalyzer(rotation='promax', n_factors=2)
fa.fit(df_cluster_30_60)
rotated_loadings = fa.loadings_

# Plot the Circle of Correlation with Promax rotation
fig, ax = plt.subplots(figsize=(8, 8))

# Plot circle
circle = plt.Circle((0, 0), 1, fill=False, edgecolor='b', linestyle='dashed')
ax.add_patch(circle)

# Plot arrows for each variable after rotation
for i, var in enumerate(df_cluster_30_60.columns):
    ax.arrow(0, 0, rotated_loadings[i, 0], rotated_loadings[i, 1],
             color='r', alpha=0.7, linewidth=0.5, head_width=0.02, head_length=0.02)
    ax.text(rotated_loadings[i, 0] * 1.15, rotated_loadings[i, 1] * 1.15, var,
            color='b', ha='center', va='center')

ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_aspect('equal', 'box')
ax.set_title('Circle of Correlation with Promax Rotation for df_cluster_30_60')

plt.show()

In [ ]:
#on remarque que l age a encore une place preponderante dans la 
#constitution des composantes principales 

# Regression logistique bis, cas multivarié avec plusieurs facteurs explicatifs, et en prenant en compte la pollution
On procède cette fois à l'échelle individuelle pour essayer de mettre plus en avant l'effet de la pollution non pas à l'échelle agrégée mais de chaque patient.

In [ ]:
data_reg = pd.read_csv("data_reg.csv")

In [ ]:
data_reg = data_reg.drop(['Unnamed: 0', 'ind', 'dept'], axis = 1)

In [ ]:
data_reg.columns

In [ ]:
data_reg_cancer = data_reg.drop(['indicatrice_Maladies cardio-neurovasculaires',
                                'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)'], axis = 1)

In [ ]:
data_reg_neuro = data_reg.drop(['indicatrice_Cancers',
                                'indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)'], axis = 1)

In [ ]:
data_reg_resp =  data_reg.drop(['indicatrice_Cancers',
                                'indicatrice_Maladies cardio-neurovasculaires'], axis = 1)

#### Régression cancer bis

In [ ]:
X = data_reg_cancer.drop('indicatrice_Cancers', axis=1)
y = data_reg_cancer['indicatrice_Cancers']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


logreg_model = LogisticRegression(class_weight={0: 1, 1: 5.5}) #Parce qu on s intersse aux cas ou la variable 
#est egale a un et si on ne balance pas comme ca, comme les 0 sont majoritaires, la regression classera tout le monde en 0


logreg_model.fit(X_train, y_train)


y_pred = logreg_model.predict(X_test)


print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))



In [ ]:
# Assuming you have a DataFrame named 'data_reg_cancer' with variables 'indicatrice_Cancers', 'feature1', 'feature2', etc.
# Replace these column names with your actual column names.

# Separate the features and target variable
X = data_reg_cancer.drop('indicatrice_Cancers', axis=1)
y = data_reg_cancer['indicatrice_Cancers']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Manually set class weights
logreg_model = LogisticRegression(class_weight={0: 1, 1: 10})

# Fit the model on the training data
logreg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg_model.predict(X_test)

# Display the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Create and train the model using the resampled data
logreg_model_resampled = LogisticRegression()
logreg_model_resampled.fit(X_resampled, y_resampled)

# Make predictions on the test data using the resampled model
y_pred_resampled = logreg_model_resampled.predict(X_test)

# Display the confusion matrix and classification report for the resampled model
print("\nConfusion Matrix (Resampled Model):")
print(confusion_matrix(y_test, y_pred_resampled))

print("\nClassification Report (Resampled Model):")
print(classification_report(y_test, y_pred_resampled))


In [ ]:
coefficients = logreg_model_resampled.coef_[0]
intercept = logreg_model_resampled.intercept_[0]

# Create a DataFrame to display the coefficients and their effects
coefficients_df = pd.DataFrame({'Variable': X.columns, 'Coefficient': coefficients})

# Calculate the odds ratio and its effect
coefficients_df['Odds Ratio'] = coefficients_df['Coefficient'].apply(lambda x: round(np.exp(x), 4))
coefficients_df['Effect'] = coefficients_df['Odds Ratio'].apply(lambda x: f"Increases by {x} times" if x > 1 else f"Decreases by {round(1/x, 4)} times")

# Display the coefficients and their effects
print(coefficients_df)

### Regression maladies respiratoires

In [ ]:
# Assuming you have a DataFrame named 'data_reg_cancer' with variables 'indicatrice_Cancers', 'feature1', 'feature2', etc.
# Replace these column names with your actual column names.

# Separate the features and target variable
X = data_reg_resp.drop('indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)', axis=1)
y = data_reg_resp['indicatrice_Maladies respiratoires chroniques (hors mucoviscidose)']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Manually set class weights
logreg_model = LogisticRegression(class_weight={0: 1, 1: 5})

# Fit the model on the training data
logreg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg_model.predict(X_test)

# Display the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Create and train the model using the resampled data
logreg_model_resampled = LogisticRegression()
logreg_model_resampled.fit(X_resampled, y_resampled)

# Make predictions on the test data using the resampled model
y_pred_resampled = logreg_model_resampled.predict(X_test)

# Display the confusion matrix and classification report for the resampled model
print("\nConfusion Matrix (Resampled Model):")
print(confusion_matrix(y_test, y_pred_resampled))

print("\nClassification Report (Resampled Model):")
print(classification_report(y_test, y_pred_resampled))


In [ ]:
coefficients = logreg_model_resampled.coef_[0]
intercept = logreg_model_resampled.intercept_[0]

# Create a DataFrame to display the coefficients and their effects
coefficients_df = pd.DataFrame({'Variable': X.columns, 'Coefficient': coefficients})

# Calculate the odds ratio and its effect
coefficients_df['Odds Ratio'] = coefficients_df['Coefficient'].apply(lambda x: round(np.exp(x), 4))
coefficients_df['Effect'] = coefficients_df['Odds Ratio'].apply(lambda x: f"Increases by {x} times" if x > 1 else f"Decreases by {round(1/x, 4)} times")

# Display the coefficients and their effects
print(coefficients_df)

### Régression maladies neurologiques et cardiaques

In [ ]:
# Assuming you have a DataFrame named 'data_reg_cancer' with variables 'indicatrice_Cancers', 'feature1', 'feature2', etc.
# Replace these column names with your actual column names.

# Separate the features and target variable
X = data_reg_neuro.drop('indicatrice_Maladies cardio-neurovasculaires', axis=1)
y = data_reg_neuro['indicatrice_Maladies cardio-neurovasculaires']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Manually set class weights
logreg_model = LogisticRegression(class_weight={0: 1, 1: 10})

# Fit the model on the training data
logreg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg_model.predict(X_test)

# Display the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Create and train the model using the resampled data
logreg_model_resampled = LogisticRegression()
logreg_model_resampled.fit(X_resampled, y_resampled)

# Make predictions on the test data using the resampled model
y_pred_resampled = logreg_model_resampled.predict(X_test)

# Display the confusion matrix and classification report for the resampled model
print("\nConfusion Matrix (Resampled Model):")
print(confusion_matrix(y_test, y_pred_resampled))

print("\nClassification Report (Resampled Model):")
print(classification_report(y_test, y_pred_resampled))

In [ ]:
coefficients = logreg_model_resampled.coef_[0]
intercept = logreg_model_resampled.intercept_[0]

# Create a DataFrame to display the coefficients and their effects
coefficients_df = pd.DataFrame({'Variable': X.columns, 'Coefficient': coefficients})

# Calculate the odds ratio and its effect
coefficients_df['Odds Ratio'] = coefficients_df['Coefficient'].apply(lambda x: round(np.exp(x), 4))
coefficients_df['Effect'] = coefficients_df['Odds Ratio'].apply(lambda x: f"Increases by {x} times" if x > 1 else f"Decreases by {round(1/x, 4)} times")

# Display the coefficients and their effects
print(coefficients_df)